In [ ]:
## 环境
import sys,  os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ["NUMBA_NUM_THREADS"] = "64"
os.environ["NUMBA_THREADING_LAYER"] = "omp"
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
sys.path.append("./mylib/")

## 基础
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

## 必要库
from threeML import *
try:
    from hawc_hal import HAL, HealpixConeROI, HealpixMapROI
except:
    from WCDA_hal import HAL, HealpixConeROI, HealpixMapROI
import mylib as my
from importlib import reload

##可选
# from threeML import silence_progress_bars, activate_progress_bars, toggle_progress_bars
# from threeML.utils.progress_bar import trange
# from tqdm import tqdm
# from astropy.io import fits
# from astropy.wcs import WCS
# from astropy.coordinates import SkyCoord
# from astropy import units as u
# from astropy.visualization import astropy_mpl_style, imshAow_norm
# from astropy.coordinates import Angle
import healpy as hp
# import traceback

## 警告
# import warnings
# np.seterr(all="ignore")
np.seterr(divide="ignore")
# warnings.simplefilter("ignore")
# silence_warnings()
# silence_logs()
# quiet_mode()
# loud_mode()
# debug_mode()

In [ ]:
#####   Data Initialize

# maptree = "../../data/KM2A_all_final.root" #old catalog
# response = "../../data/KM2A_DR_all.root"

# maptree = "../../data/KM2A1234full_skymap_rcy.root"
# # maptree = "/data/home/cwy/Science/KM2A1234full_skymap_rcy_test.root"
# response = "../../data/KM2A1234full_mcpsf_DRfinal.root"

maptree = "../../data/KM2A_20240731_xsq_out.root"
response = "../../data/KM2A_DR_xsq.root"

region_name="S147"
if not os.path.exists(f'../res/{region_name}/'):
    os.system(f'mkdir ../res/{region_name}/')
# ra_S147, dec_S147 =  84.60, 28.29
ra_S147, dec_S147 =   83, 26 #84.60, 28.29
ra1,dec1=ra_S147, dec_S147

data_radius = 7.0
model_radius = 8.0

roimap = my.getmaskedroi(ra1, dec1, data_radius, [(85.78, 23.40, 3), (83.63, 22.02, 3)]) #
roi = HealpixMapROI(ra=ra1,dec=dec1, data_radius=data_radius, model_radius=model_radius, roimap=roimap) 

# roi=HealpixConeROI(ra=ra1,dec=dec1,data_radius=data_radius,model_radius=model_radius) 

KM2A = HAL("KM2A", maptree, response, roi, flat_sky_pixels_size=0.17)

#####   Data Situation
%matplotlib inline
KM2A.set_active_measurements(0, 13)
KM2A.display()

fig = KM2A.display_stacked_image(smoothing_kernel_sigma=0.25)
fig.show()
fig.savefig(f"../res/{region_name}/counts_all.png",dpi=300)

In [ ]:
my.Search(ra1, dec1, data_radius, model_radius, region_name, KM2A, roi, 4,13, fromcatalog=1, detector="KM2A", fixcatall=0)

In [ ]:
Modelname="1pt_freeDGE_4-13_KM2A"
if not os.path.exists(f'../res/{region_name}/{Modelname}/'):
    os.system(f'mkdir ../res/{region_name}/{Modelname}/')

lm = my.getcatModel(ra1, dec1, data_radius, model_radius, detector="KM2A", rtsigma = 8, rtflux = 15, rtindex = 2, rtp = 8)

# S147 = my.set_diffusemodel("S147", '/data/home/cwy/Science/3MLWCDA/Standard/res/S147/S147_HI_temp.fits', K=7.3776826e-14, Kb=(1e-17, 1e-12))

PWN = my.setsorce("PWN",84.5,28.2, k=1e-14, index=-3.6, #sigma=0.2, sb=(0,5), #sf=True, #raf=True,decf=True,
                indexb=(-4,-1),kb=(1e-19, 1e-14), #indexf=True,
                fitrange=0.5, piv=50)

Diffuse = my.set_diffusebkg( 
                ra1, dec1, data_radius, data_radius, K=1e-15, Kb=(1e-19, 1e-13), index=-3.2,
                Kf=False, indexf=False, piv=50
                )
# lm = Model(PWN, Diffuse) #S147,
# lm = Model(S147, PWN, Diffuse)
# lm.add_source(S147)
lm.add_source(PWN); lm.add_source(Diffuse)

lm.save(f"../res/{region_name}/{Modelname}/Model_init.yml", overwrite=True)
with open(f'../res/{region_name}/{Modelname}/Model_init.txt', 'w') as f:
    for key, value in lm.parameters.items():
        f.write(f'{key}: {value}\n')

lm.display(complete=True)

In [ ]:
%matplotlib inline
result = my.fit(region_name, Modelname, KM2A, lm, 4, 13,mini="ROOT")
sources = my.get_sources(lm,result)
result[1][0]
# result[0].get_contours(J0248.Gaussian_on_sphere.lon0,39.88,44.88,30)

In [ ]:
resu = my.getresaccuracy(KM2A, lm, savepath = f"../res/{region_name}/{Modelname}/KM2A_sres.fits")
new_source_idx = np.where(resu==np.ma.max(resu))[0][0]
new_source_lon_lat=hp.pix2ang(1024,new_source_idx,lonlat=True)
print(new_source_lon_lat)

plt.figure()
hp.gnomview(resu,norm='',rot=[ra1,dec1],xsize=200,ysize=200,reso=6,title=Modelname)
plt.scatter(new_source_lon_lat[0],new_source_lon_lat[1],marker='x',color='red')
plt.show()
plt.savefig(f"../res/{region_name}/{Modelname}KM2A_res.png",dpi=300)

In [ ]:
TSlist = ["PWN", "Diffuse"] #"S147",
TS, TSresults = my.getTSall(TSlist, region_name, Modelname, result, KM2A)
TSresults

In [ ]:
%matplotlib inline
sources = my.get_sources(lm,result)
sources.pop("Diffuse")

# sources={}
map2, skymapHeader = hp.read_map("../../data/fullsky_KM2A_20240131_3.5.fits.gz",h=True)
map2 = my.maskroi(map2, roi)
fig = my.drawmap(region_name, Modelname, sources, map2, ra1, dec1, rad=2*data_radius, contours=[10000],save=1, 
                # cat={ }, #
                cat={ "LHAASO": [0, "P"],"TeVCat": [0, "s"],"PSR": [0, "*"],"SNR": [0, "o"],"3FHL": [0, "D"],"4FGL": [0, "d"],"YMC": [0, "^"],"GYMC": [0, "v"],"WR": [0, "X"],"size": 20,"markercolor": "grey","labelcolor": "black","angle": 60,"catext": 1 },
                # color="Milagro"
                  color="Fermi"
                  # Drawdiff=1,legend=0
                  )

In [ ]:
my.write_resmap(region_name, Modelname, KM2A, roi, maptree, ra1, dec1, "J0057_res_all",[],[0,0])

In [ ]:
# Flux_KM2A0, jls0  = my.getdatapoint(KM2A, lm, maptree, response, roi, "S147", ifpowerlawM=1, piv=50)
Flux_KM2A1, jls1  = my.getdatapoint(KM2A, lm, maptree, response, roi, "Diffuse", ifpowerlawM=1, piv=50, scanbin=0)
Flux_KM2A2, jls2  = my.getdatapoint(KM2A, lm, maptree, response, roi, "PWN", ifpowerlawM=1, piv=50, scanbin=0)

In [ ]:
%matplotlib inline
import matplotlib as mpl

x_Max=10000.
x_Min=1
y_Min=0.8e-15
y_Max=2e-9
fig,ax = plt.subplots()
plot_spectra(
    result[0].results,
    sources_to_use=["PWN","Diffuse"], #,"Diffuse","ext1","ext2","ext4","ext5","ext6","ext7"| "S176",
    include_extended=True,
    ene_min=x_Min,
    ene_max=x_Max,
    num_ene=30,
    energy_unit="TeV",
    flux_unit="TeV/(s cm2)",
    subplot=ax,
    )

# my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A0,"S147","tab:red", usexerr=True)
my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A2,"PWN","tab:red", usexerr=True)
my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A1,"Diffuse","tab:blue", usexerr=True)
# my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/1pt+HI_freeDGE_0-5_roi6/Spectrum_PWN.txt",label="PWN WCDA",color="tab:purple", threshold=1)

# LHAASO_sensitivity = np.genfromtxt("./LHAASO_sensitivity.csv",delimiter=',')
# plt.plot(LHAASO_sensitivity[:,0],LHAASO_sensitivity[:,1]*1e-11,label="LHAASO 1yr", c="black")

ax.set_xlim(x_Min,x_Max)
ax.set_ylim(y_Min,y_Max)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r"$E^2\,dN/dE$ [TeV cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Energy [TeV]")
plt.legend()
plt.savefig(f'../res/{region_name}/{Modelname}/Spectrum.png', dpi=300)
plt.savefig(f'../res/{region_name}/{Modelname}/Spectrum.pdf')

In [ ]:
%matplotlib inline
import matplotlib as mpl

x_Max=10000.
x_Min=0.1
y_Min=0.8e-16
y_Max=2e-11
fig,ax = plt.subplots()

# my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A0,"S147","tab:red", usexerr=True)
# my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A2,"PWN","tab:red", usexerr=True)
# my.Draw_sepctrum_points(region_name, Modelname, Flux_KM2A1,"Diffuse","tab:blue", usexerr=True)
# my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/1pt+HI_freeDGE_0-5_roi6/Spectrum_PWN.txt",label="PWN WCDA",color="tab:purple", threshold=1)

my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/cat+1ext+1pt_freeDGE_0-6/Spectrum_PWN.txt",label="PWN WCDA",color="tab:blue", threshold=1)
my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/cat+1ext+1pt_freeDGE_0-6/Spectrum_S147.txt",label="S147",color="tab:green", threshold=1)
my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/cat+1pt_freeDGE_4-13_roi6_KM2A/Spectrum_PWN.txt",label="PWN KM2A", color="blue", threshold=1)

# my.Draw_spectrum_fromfile("/data/home/cwy/Science/3MLWCDA/Standard/res/S147/cat+1ext+1pt_freeDGE_0-6/Spectrum_PWN.txt",label="PWN WCDA",color="tab:purple", threshold=1)


# LHAASO_sensitivity = np.genfromtxt("./LHAASO_sensitivity.csv",delimiter=',')
# plt.plot(LHAASO_sensitivity[:,0],LHAASO_sensitivity[:,1]*1e-11,label="LHAASO 1yr", c="black")

ax.set_xlim(x_Min,x_Max)
ax.set_ylim(y_Min,y_Max)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r"$E^2\,dN/dE$ [TeV cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Energy [TeV]")
plt.legend()
plt.savefig(f'../res/{region_name}/{Modelname}/Spectrum.png', dpi=300)
plt.savefig(f'../res/{region_name}/{Modelname}/Spectrum.pdf')